## Проект "A/B-тестирование".

## Описание проекта.
Задача — провести оценку результатов A/B-теста. В нашем распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.

Необходимо оценить корректность проведения теста и проанализировать его результаты.

Дополнитьельные задачи:

* удостоверьиться, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно;
* проверить равномерность распределения пользователей по тестовым группам и правильность их формирования.

## План выполнения работы.

- Опишите цели исследования
- Исследуйте данные:
    - Требуется ли преобразование типов?
    - Опишите природу пропущенных значений и дубликатов, если их обнаружите.
- Оцените корректность проведения теста. Обратите внимание на:
    - Соответствие данных требованиям технического задания. Проверьте корректность всех пунктов технического задания.
    - Время проведения теста. Убедитесь, что оно не совпадает с маркетинговыми и другими активностями.
    - Аудиторию теста. Удостоверьтесь, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверьте равномерность распределения по тестовым группам и правильность их формирования.
- Проведите исследовательский анализ данных:
    - Количество событий на пользователя одинаково распределены в выборках?
    - Как число событий в выборках распределено по дням?
    - Как меняется конверсия в воронке в выборках на разных этапах?
    - Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?
- Оцените результаты A/B-тестирования
    - Что можно сказать про результаты A/В-тестирования?
    - Проверьте статистическую разницу долей z-критерием.
- Опишите выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования. Сделайте общее заключение о корректности проведения теста.

In [ ]:
# импортирую библиотеки
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
from scipy import stats as st 
import datetime as dt
import plotly.express as px
from plotly import graph_objects as go
import numpy as np
import math as mth
from scipy import stats as st

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Шаг 1. Ознакомление с данными и предобработка.

In [ ]:
# загружаю данные
try:
    events = pd.read_csv(r"D:\DOCS\datasets\Финальный проект\AB тест/final_ab_events.csv", parse_dates=['event_dt'])
    users = pd.read_csv(r"D:\DOCS\datasets\Финальный проект\AB тест/final_ab_new_users.csv", parse_dates=['first_date'])
    participants = pd.read_csv(r"D:\DOCS\datasets\Финальный проект\AB тест/final_ab_participants.csv")
    marketing = pd.read_csv(r"D:\DOCS\datasets\Финальный проект\AB тест/ab_project_marketing_events.csv", parse_dates=['start_dt','finish_dt'])
except FileNotFoundError:
    events = pd.read_csv('/datasets/final_ab_events.csv', parse_dates=['event_dt'])
    users = pd.read_csv('/datasets/final_ab_new_users.csv', parse_dates=['first_date'])
    participants = pd.read_csv('/datasets/final_ab_participants.csv')
    marketing = pd.read_csv('/datasets/ab_project_marketing_events.csv', parse_dates=['start_dt','finish_dt'])

In [ ]:
# написал функцию, выводящую основную информацию об имеющихся датафреймах
def basic_information(x):
    display(x.head(10));
    x.info();
    print('*'*50)
    print('Дубликатов',x.duplicated().sum());
    print('*'*50);
    print('Количество пропусков');
    display(x.isna().sum());

In [ ]:
# применил функцию basic_information к датафрейму events
basic_information(events)

Структура файла:

- `user_id` — идентификатор пользователя;
- `event_dt` — дата и время события;
- `event_name` — тип события;
- `details` — дополнительные данные о событии. Например, для покупок, `purchase,` в этом поле хранится стоимость покупки в долларах.

In [ ]:
# применил функцию basic_information к датафрейму users
basic_information(users)

Структура файла:

- `user_id` — идентификатор пользователя;
- `first_date` — дата регистрации;
- `region` — регион пользователя;
- `device` — устройство, с которого происходила регистрация.

In [ ]:
# применил функцию basic_information к датафрейму participants
basic_information(participants)

Структура файла:

- `user_id` — идентификатор пользователя;
- `ab_test` — название теста;
- `group` — группа пользователя.

In [ ]:
# применил функцию basic_information к датафрейму marketing
basic_information(marketing)

Структура файла:

- `name` — название маркетингового события;
- `regions` — регионы, в которых будет проводиться рекламная кампания;
- `start_dt` — дата начала кампании;
- `finish_dt` — дата завершения кампании.

При анализе информации о представленных датафреймах и об их содержании выявлены пропуски в столбце details датафрейма events. Рассмотри пропуски более детально.

In [ ]:
# вывел количество разных типов событий, содержащихся в столбще event_name датафрейма events
events_group = events.groupby('event_name').agg({'event_name':'count'})
events_group.rename(columns={'event_name': 'event_count'}, inplace=True)
events_group.sort_values(by='event_count', ascending=False)

По количеству выполнения события распределяются следующим образом login → product_page → purchase → product_cart
* login - регистрация,
* product_page - просмотр товара,
* product_cart - просмотр корзины,
* purchase - покупка.

### Вывод 
* Выполнено преобразование столбца event_dt датафрейма events, столбца first_date датафрейма users, столбцов start_dt и finish_dt даиафрейма marketing к формату datetime64. 
* Явных дубликатов ни в одном из исследуемых датафреймов не выявлено
* Выявлены пропуски в столбце details датафрейма events. Пропуски в данном столбце полностью отсутствуют при выполнении пользователями эвента purchase. Значениями в данном случае являются суммы покупки. В других типах событий (product_page, product_cart, login) значения в солбце details полностью отсутствуют, вероятно, это связано с отсутствием какой-либо уточняющей информации для данных типов событий. В заполнении пропусков необходимости нет. Решено оставить их так как они не повлияют на дальнейший анализ.
* Количество событий в исследуемом логе представлено следующим образом login(вход на сайт) → product_page(страница товара) → purchase(покупка) → product_cart(страница корзины). БОльшее количество эвента purchase в сравнении с эвентом product_cart, вероятно, связано с тем, что неколторые пользователи переходят к оплате товара сказу со страницы товара не переходя в корзину.

## Шаг 2. Оценка корректности проведения теста.

In [ ]:
# вывожу количество уникальных групп в датафрейме participants
print('Количество уникальных групп в колонке group датафрейма participants равно:', participants['group'].nunique())

In [ ]:
# вывожу количество пуникальных пользователей, сгруппированных по названию теста и группе в тесте
participants_group = participants.groupby(['ab_test','group']).agg({'user_id':'nunique'})
participants_group

В датафрейме participants выявлено два теста: interface_eu_test, recommender_system_test. Согласно техническому заданию нам необходимо анализировать recommender_system_test

In [ ]:
# определил пользователей, участвующих в тесте recommender_system_test
data_rst = participants.loc[participants['ab_test'].values == 'recommender_system_test']
data_rst

В тесте принимает участие 6701 пользователь. Выше, чем по техническому заданию ( по техническому заданию предполагаемое количество участников теста 6000).
Соотношение исследуемых групп: А - 3824 пользователя B - 2877 пользователей.

In [ ]:
# определил пользователей участвующих в тесте interface_eu_test
data_iet = participants.loc[participants['ab_test'].values == 'interface_eu_test']
data_iet

In [ ]:
# определил пользователей, попавших в оба теста
users_in_tests = pd.merge(data_rst, data_iet, on=['user_id'], how='inner')
users_in_tests = users_in_tests['user_id']
users_in_tests

Выявлено 1602 пользователя, вошедшие в оба теста. Необходимо удалить их из анализа, так как достоверно определить который из тестов повлиял на поведение пользователей не представляется возможным.

In [ ]:
# удалил пользователей, попавших в оба теста из анализа recommender_system_test
data_rst = data_rst.query('user_id not in @users_in_tests')
data_rst

In [ ]:
rst_group = data_rst.groupby('group').agg({'user_id':'nunique'})
rst_group

После удаления пользователей, вошедших в оба теста, в тесте recommender_system_test осталось 5099 пользователей. По группам пользователи были поделены в следующем порядке: А- 2093 пользователя, В - 2196 пользователей.

In [ ]:
print('Дата запуска теста', events['event_dt'].dt.date.min())
print('Дата завершения теста', events['event_dt'].dt.date.max())

В техническом задании дата запуска теста определена дата запуска: 2020-12-07, что соответствует началу логирования эвентов пользователей. Дата остановки теста согласно техническому заданию 2021-01-04, однако, логирование событий ограничено 2020.12.30.

In [ ]:
# добавил к информацию о дате регистрации пользователя его регионе и устройстве
users_test = data_rst.merge(users, how='left', on='user_id')
users_test

In [ ]:
# сгруппировал количество всех пользователей по региону регистрации
users_group = users.groupby('region').agg({'user_id':'nunique'})
users_group.rename(columns={'user_id': 'users_nunique'}, inplace=True)
users_group = users_group.sort_values(by='users_nunique', ascending=False)

# сгруппировал количество пользователей, участвующих в тесте, по региону регистрации
rst_users_group = users_test.groupby('region').agg({'user_id':'nunique'})
rst_users_group.rename(columns={'user_id': 'rst_users_nunique'}, inplace=True)
rst_users_group = rst_users_group.sort_values(by='rst_users_nunique', ascending=False)

# определил процент пользователей участвующих в тесте от общего количества пользователей в различных регионах
users_group = users_group.merge(rst_users_group, how='inner', on='region')
users_group['percent'] = round((users_group['rst_users_nunique'] / users_group['users_nunique'] * 100),2)
users_group

Процент пользователей участвующих в тесте от общего количества пользователей из Европы составляет 10.26% согласно техническому заданию в тесте должно быть отобрано 15% новых пользователей из региона EU.

In [ ]:
print('Дата начала набора пользователей', users_test['first_date'].dt.date.min())
print('Дата завершения набора пользователей', users_test['first_date'].dt.date.max())

Дата завершения набора новых пользователей на исследуемый тест - 2020-12-21, что соответствует техническому заданию.

In [ ]:
# добавил информацию об активностях пользователей
users_test = users_test.merge(events, on='user_id', how='left') 
users_test

In [ ]:
# определил какие маркетинговые мобытия попали в даты проведения теста
marketing_query = marketing.query('finish_dt >= "2020-12-07" and start_dt <="2021-01-04"')
marketing_query

В период проведения теста попала акция Christmas&New Year Promo. Данная акция может отразиться на результатах тестирования, так как проводилась в том числе и в самом многочисленном регионе тестирования EU. Необходимо исключить из исследования информацию об активности пользователей после 2020-12-25.

In [ ]:
users_test = users_test.query('event_dt < "2020-12-25"')
users_test

In [ ]:
users_test['event_date'] = users_test['event_dt'].dt.date
users_test['event_date'] = pd.to_datetime(users_test['event_date'])
users_test['lifetime'] = (users_test['event_date'] - users_test['first_date']).dt.days
users_test = users_test[users_test['lifetime'] <= 14]
users_test

### Вывод
* При анализе выявлено 1602 пользователя, вошедшие в оба теста. Принято решение удалить их из анализа, так как достоверно определить который из тестов повлиял на поведение пользователей не представляется возможным.
* После удаления пользователей, вошедших в оба теста, в тесте recommender_system_test осталось 5099 пользователей. По группам пользователи были поделены в следующем порядке: А- 2093 пользователя, В - 2196 пользователей.
* В техническом задании дата запуска теста определена дата запуска: 2020-12-07, что соответствует началу логирования эвентов пользователей. Дата остановки теста согласно техническому заданию 2021-01-04, однако, логирование событий ограничено 2020-12-30.
* Дата завершения набора новых пользователей на исследуемый тест - 2020-12-21, что соответствует техническому заданию.
* В период проведения теста попала акция Christmas&New Year Promo. Данная акция может отразиться на результатах тестирования, так как проводилась в том числе и в самом многочисленном регионе тестирования EU. Принято решение исключить из исследования информацию об активности пользователей после 2020-12-25.
* Процент пользователей участвующих в тесте от общего количества пользователей из Европы после всех преодбразований составил 10,26%. Согласно техническому заданию в тесте должно быть отобрано 15% новых пользователей из региона EU.

## Шаг 3. Исследовательский анализ данных.

In [ ]:
# определил как события распределены между исследуемыми группами
ab_events = users_test.groupby('group').agg({'event_name':'count'})
ab_events

Количество событий в выборке А значительно превосходит количество событий в выборке В.

In [ ]:
# определил количесвто событий для каждого пользователя
ab_events_group = users_test.groupby(['user_id', 'group']).agg({'event_name':'count'}).reset_index()


In [ ]:
fig = sns.set(style="whitegrid", palette="pastel", color_codes=True) 
sns.boxplot(data=ab_events_group, y='event_name', x='group')
plt.title('Количество эвентов на пользователя по группам', fontsize=16)
plt.xlabel('Группа', fontsize = 13)
plt.ylabel('Количество эвентов', fontsize = 13);

В группе А пользователь в среднем совершает 6 действий. В группе В -4 действия.

In [ ]:
# определил распределение событий в группах по дням
users_test_data_pivot = users_test.pivot_table(index='event_date', columns='group', values='user_id', aggfunc='count')
users_test_data_pivot.head(5)

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
users_test_data_pivot.plot(kind='bar', grid=True, width = 0.5, figsize=(15,5));
plt.xlabel('Дата эвента', fontsize = 15)
plt.ylabel('Количество эвентов', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество эвентов в разбивке по дате', fontsize = 15)
plt.show()

Из диаграммы следует, что вплеск активности пользователей в группе А пришелся на 14 - 21 декабря.С максимальной активностью 21 декабря, когда было зарегистрировано около 1550 событий. Показатели активности в группе В также имеют самый высокий показатель 21 декабря, когда было совершено около 370 событий. 

In [ ]:
data_ev_pivot = (users_test.pivot_table(index='event_name',values='user_id',aggfunc='nunique')
                    .sort_values(by='user_id', ascending=False)).reset_index()
data_ev_pivot 

In [ ]:
# Построил воронку процента прохождения к последующему этапу от общего количества пользователей
fig = go.Figure(go.Funnel(
    y = data_ev_pivot['event_name'],
    x = data_ev_pivot['user_id'],
    textposition = "inside",
    textinfo = "value+percent initial"
    ))
fig.update_layout(title='Общая воронка событий', title_x=0.5)
fig.show()

Наибольшее количество пользователей теряется на этапе перехода со страницы просмотра товара(product_page) на страницу просмотра корзины(product_cart)	

In [ ]:
users_test_group_pivot = users_test.pivot_table(index='event_name', 
                                                values='user_id', 
                                                columns='group', 
                                                aggfunc='nunique').sort_values('A', ascending=False).reset_index()   
users_test_group_pivot 

In [ ]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'A',
    y = users_test_group_pivot ['event_name'],
    x = users_test_group_pivot ['A'],
    textinfo = "value + percent initial"))

fig.add_trace(go.Funnel(
    name = 'B',
    y = users_test_group_pivot ['event_name'],
    x = users_test_group_pivot ['B'],
    textinfo = "value + percent initial"))

fig.update_layout(title='Воронка событий для каждой группы пользователей', title_x=0.5)
fig.show()

Конверсия в покупку в группе А (31%) выше, чем в группе В (28%).

### Вывод
* Было определено, что количество событий в выборке А (12635 событий) значительно превосходит количество событий в выборке В (3665 событий).
* В группе А пользователь в среднем совершает 6 действий. В группе В - 4 действия.
* Общее количество событий, в разбикве по дням, в группе А выше, чем в группе В. Максимальная активность пользователей наблюдается 21.12.2020.
* Наибольшее количество пользователей в обеих группах теряется на этапе перехода со страницы просмотра товара(product_page) на страницу просмотра корзины(product_cart)
* Есть пользователи, которые пропускают страницу корзины (product_cart) и переходят со страницы товара (product_page) сразу к оплате (purchase). В связи с этип общее количество пользователей совершивших оплату выше, чем общее количество пользователей, перешедших в корзину.
* Конверсия в покупку в группе А (31%) выше, чем в группе В (28%).

## Шаг 4. Оценка результатов A/B-тестирования

In [ ]:
# распределение обзего количества событий между группами
ab_events

In [ ]:
# количество выполнения каждого события в разбивке по группам
users_test_group_pivot 

In [ ]:
# функция для определения статистической значимости различий между группами
def z_test(actA,actB,groupA,groupB, eventname,alpha):
    
    p1 = actA/groupA
    p2 = actB/groupB

    p_combined = (actA + actB) / (groupA + groupB)
    # считаем статистику в ст.отклонениях стандартного нормального распределения
    difference = p1 - p2
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/groupA + 1/groupB))
    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)
    p_value =(1 - distr.cdf(abs(z_value))) * 2

    print(f'{eventname} p-значение: ', p_value)
    
    bonferroni_alpha =   alpha / 4

    if p_value < bonferroni_alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
        print('**************')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')
        print('**************')

#### Формулирую гипотезы

* H0 Доли уникальных посетителей, побывавших на этапе воронки, одинаковы.
* H1 Между долями уникальных посетителей, побывавших на этапе воронки, есть значимая разница.

In [ ]:
test = [z_test(users_test_group_pivot ['A'][i],users_test_group_pivot['B'][i],ab_events['event_name'][0],ab_events['event_name'][1],users_test_group_pivot['event_name'][i],alpha = .05)
        for i in range(len(users_test_group_pivot['A']))]

### Вывод
* Выявлена значимая разница в количестве уникальных посетителей между группами только на этапе login. На остальных этапах доли пользователей в обеих группах оказались равны.

### Тест можно считать завершенным, результат теста отрицательный. Улучшение каждой метрики не менее, чем на 10% в новой платёжной воронке не выявлено.

## Шаг 5. Общий вывод.

### Ознакомление
* Выполнено преобразование столбца event_dt датафрейма events, столбца first_date датафрейма users, столбцов start_dt и finish_dt даиафрейма marketing к формату datetime64. 
* Выявлены пропуски в столбце details датафрейма events. Решено оставить их так как они не повлияют на дальнейший анализ.


### Оценка корректности
* При анализе выявлено 1602 пользователя, вошедшие в оба теста. Принято решение удалить их из анализа, так как достоверно определить который из тестов повлиял на поведение пользователей не представляется возможным.
* После удаления пользователей, вошедших в оба теста, в тесте recommender_system_test осталось 5099 пользователей. По группам пользователи были поделены в следующем порядке: А- 2093 пользователя, В - 2196 пользователей.
* Дата остановки теста согласно техническому заданию 2021-01-04, однако, логирование событий ограничено 2020-12-30.
* В период проведения теста попала акция Christmas&New Year Promo. Данная акция может отразиться на результатах тестирования, так как проводилась в том числе и в самом многочисленном регионе тестирования EU. Принято решение исключить из исследования информацию об активности пользователей после 2020-12-25.
* Процент пользователей участвующих в тесте от общего количества пользователей из Европы после всех преодбразований составил 10,26%. Согласно техническому заданию в тесте должно быть отобрано 15% новых пользователей из региона EU.

### Исследовательский анализ
* Было определено, что количество событий в выборке А (12635 событий) значительно превосходит количество событий в выборке В (3665 событий).
* В группе А пользователь в среднем совершает 6 действий. В группе В - 4 действия.
* Наибольшее количество пользователей в обеих группах теряется на этапе перехода со страницы просмотра товара(product_page) на страницу просмотра корзины(product_cart)
* Конверсия в покупку в группе А (31%) выше, чем в группе В (28%).

### Результат
* Выявлена значимая разница в количестве уникальных посетителей между группами только на этапе login. На остальных этапах доли пользователей в обеих группах оказались равны.

### Тест можно считать завершенным, результат теста отрицательный. Улучшение каждой метрики не менее, чем на 10% в новой платёжной воронке не выявлено.